In [1]:
import pennylane as qml
from pennylane import numpy as qnp

In [2]:
# Number of qubits and layers
num_wires = 4
num_layers = 2

In [3]:
dev = qml.device("default.qubit", wires=num_wires)


In [4]:
@qml.qnode(dev)
def quantum_model(params):
    # for i in range(num_wires):
    #     qml.Hadamard(wires=i)

    qml.StronglyEntanglingLayers(weights=params, wires=range(num_wires))
    qml.StronglyEntanglingLayers(weights=params, wires=range(num_wires), imprimitive=qml.ops.CZ)
    # qml.StronglyEntanglingLayers(weights=params, wires=range(num_wires), ranges=[2, 3], imprimitive=qml.ops.CZ)

    
    return qml.probs(wires=range(num_wires))




In [5]:
import numpy as np



# Generate random initial parameters
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_wires)
weights = qnp.random.random(size=shape,requires_grad=True)
initial_params = weights
print(initial_params)
print(quantum_model(initial_params))


[[[0.71057357 0.77621081 0.76538924]
  [0.72051125 0.41137903 0.32160405]
  [0.90629772 0.64285325 0.2766322 ]
  [0.76823308 0.11551333 0.36907921]]

 [[0.99386255 0.55425412 0.75601916]
  [0.11278393 0.35506369 0.13033963]
  [0.39912154 0.79066149 0.47446638]
  [0.82703607 0.07259491 0.63490905]]]
[0.29921804 0.04542749 0.18240868 0.0058925  0.02391837 0.0012155
 0.10680671 0.00371639 0.12981903 0.00659569 0.03822193 0.00670397
 0.08312951 0.06027082 0.00379868 0.00285668]


In [6]:
def objective_cost(params, target_probs):
    predicted_probs = quantum_model(params)
    loss= qml.math.sum(qml.math.abs(predicted_probs - target_probs)**2)
    return loss

In [7]:
num_steps = 800

np.random.seed(100)

# Optimization using gradient-based methods
opt = qml.GradientDescentOptimizer(stepsize=0.1)
params = initial_params  # Initial parameters

# Create a random target probability distribution
target_probs = qnp.random.rand(2**num_wires,requires_grad=False)
target_probs /= np.sum(target_probs)  # Normalize to ensure it's a valid probability distribution


target_probs = target_probs  # The target probability distribution

for step in range(num_steps):
    # print(params)
    # params, cost = opt.step_and_cost(objective_cost, params,target_probs)
    params, cost = opt.step_and_cost(lambda p: objective_cost(p, target_probs), params)
    
    if(step%10 == 0):
        print('step: ',step,'cost: ',cost)



print('================ target ',target_probs)
print('================ predicted ',quantum_model(params))

step:  0 cost:  0.14201131747084153
step:  10 cost:  0.11693393554179493
step:  20 cost:  0.10805512058487056
step:  30 cost:  0.1019006520550889
step:  40 cost:  0.09661447724486966
step:  50 cost:  0.09175759663966568
step:  60 cost:  0.08717256284760685
step:  70 cost:  0.08272803027356039
step:  80 cost:  0.07824720822340414
step:  90 cost:  0.07350570291099012
step:  100 cost:  0.06827245315378763
step:  110 cost:  0.06239657270663858
step:  120 cost:  0.055926816301384755
step:  130 cost:  0.04918175021539445
step:  140 cost:  0.04266067903073724
step:  150 cost:  0.03682415266929272
step:  160 cost:  0.03192675187930741
step:  170 cost:  0.02800320855941573
step:  180 cost:  0.024945971404771818
step:  190 cost:  0.022588679632611616
step:  200 cost:  0.020761896712340237
step:  210 cost:  0.01932057463485301
step:  220 cost:  0.01815217237631485
step:  230 cost:  0.017174542907688356
step:  240 cost:  0.016329994589126172
step:  250 cost:  0.015578975792913079
step:  260 cost: 